In [ ]:
using LinearAlgebra
using Polynomials

In [4]:
# метод Гершгорина
function gershgorin_intervals(A::Matrix{Float64})
    n = size(A, 1)
    
    intervals = Vector{Tuple{Float64, Float64}}()
    
    for i in 1:n
        # Диагональный элемент a_ii - центр круга Гершгорина
        a_ii = A[i, i]
        
        # Сумма модулей внедиагональных элементов в строке i - радиус круга r_i
        # r_i = ∑|a_ij|, j≠i
        r_i = sum(j -> j != i ? abs(A[i, j]) : 0.0, 1:n)
        
        # Интервал для i-го круга Гершгорина: [a_ii - r_i, a_ii + r_i]
        # теорема Гершгорина: S_i = {z ∈ C : |z - a_ii| ≤ r_i}
        interval = (a_ii - r_i, a_ii + r_i)
        push!(intervals, interval)
    end
    
    # объединение всех интервалов
    # Все собственные значения лежат в объединении кругов Гершгорина
    min_bound = minimum(first.(intervals))
    max_bound = maximum(last.(intervals))
    overall_interval = (min_bound, max_bound)
    
    return intervals, overall_interval
end


gershgorin_intervals (generic function with 1 method)

In [5]:
# Проверка интервалов Гершгорина
function check_gershgorin_intervals(λs::Vector{Float64}, intervals::Vector{Tuple{Float64,Float64}}, overall_interval::Tuple{Float64,Float64}, tol::Float64)
    all_in_overall = true
    for (i, λ) in enumerate(λs)
        in_overall = (overall_interval[1] - tol <= λ <= overall_interval[2] + tol)
        all_in_overall &= in_overall
    end
    
    for (i, λ) in enumerate(λs)
        in_specific_intervals = Vector{Tuple{Float64,Float64}}()
        interval_indices = Int[]
        
        for (j, (L, R)) in enumerate(intervals)
            if L - tol <= λ <= R + tol
                push!(in_specific_intervals, (L, R))
                push!(interval_indices, j)
            end
        end
        
        if !isempty(in_specific_intervals)
            intervals_str = join(["Δ$idx=[$(round(L, digits=4)), $(round(R, digits=4))]" 
                                 for (idx, (L, R)) in zip(interval_indices, in_specific_intervals)], ", ")
            println("λ$i = $(round(λ, digits=6)) ∈ {$intervals_str}")
        else
            println("λ$i = $(round(λ, digits=6)) не принадлежит ни одному интервалу!")
        end
    end
    println()
    
    return all_in_overall
end

check_gershgorin_intervals (generic function with 1 method)

In [6]:
 # Проверка следа
function check_trace(A::Matrix{Float64}, λs, tol)
    tr_A = tr(A)
    sum_λ = sum(λs)
    tr_error = abs(tr_A - sum_λ)
    return tr_error < tol
    
end

check_trace (generic function with 1 method)

In [7]:
# Проверка определителя
function check_determinant(A::Matrix{Float64}, λs, tol)
    det_A = det(A)
    prod_λ = prod(λs)
    det_error = abs(det_A - prod_λ)
    return det_error < tol
end

check_determinant (generic function with 1 method)

In [8]:
function checks(A::Matrix{Float64}, λs, tol)
    trace_check = check_trace(A, λs, tol)
    determ_check = check_determinant(A, λs, tol)
    intervals, interval = gershgorin_intervals(A)
    gersh_check = check_gershgorin_intervals(λs, intervals, interval, tol)
    
    println("Результаты проверок:")
    println("След матрицы: $trace_check")
    println("Определитель: $determ_check")
    println("Интервалы Гершгорина: $gersh_check")
    
    all_checks_passed = trace_check && determ_check && gersh_check
    println("Все проверки пройдены: $all_checks_passed")
end

checks (generic function with 1 method)

In [9]:
# метод А. М. Данилевского
function danilevsky_method_symmetric(A::Matrix{Float64})
    n = size(A, 1)
    
    # Проверка симметричности
    if !isapprox(A, A')
        error("Матрица не симметрична!")
    end
    
    # B_total - матрица подобия
    # B = B₁ ⋅ B₂ ⋅ ... ⋅ B_{n-1}
    B_total = Matrix{Float64}(I, n, n)
    
    # P - преобразуемая матрица, начинается с исходной матрицы A
    # В конечном итоге должна принять форму Фробениуса
    P = copy(A)
    
    # n-1 преобразований подобия
    for k in n-1:-1:1
        # Проверка на нулевой элемент
        if abs(P[k+1, k]) < 1e-10
            error("Нулевой элемент на позиции ($(k+1), $k)")
        end
        
        # Построение матрицы B_k
        B_k = Matrix{Float64}(I, n, n)
        for j in 1:n
            if j != k
                # b_{k,j} = -P[k+1, j] / P[k+1, k] для j ≠ k
                # формула: b_{n-1,j} = -a_{nj}/a_{n,n-1} для j ≠ n-1
                B_k[k, j] = -P[k+1, j] / P[k+1, k]
            else
                # b_{k,k} = 1 / P[k+1, k]
                # формула: b_{n-1,n-1} = 1/a_{n,n-1}
                B_k[k, j] = 1.0 / P[k+1, k]
            end
        end
        
        # Построение обратной матрицы B_k^{-1}
        B_k_inv = Matrix{Float64}(I, n, n)
        # k-я строка B_k^{-1} равна (k+1)-й строке матрицы P
        B_k_inv[k, :] = P[k+1, :]
        
        # Преобразование подобия: P = B_k^{-1} ⋅ P ⋅ B_k
        P = B_k_inv * P * B_k
        
        # Накопление матрицы подобия: B_total = B_total ⋅ B_k
        B_total = B_total * B_k
    end
    
    # Характеристическое уравнение: λⁿ - p₁λⁿ⁻¹ - p₂λⁿ⁻² - ... - p_n = 0
    # Коэффициенты берутся из первой строки матрицы Фробениуса P
    coefficients = [1.0; -P[1, :]]

    # Нахождение собственных векторов
    println("Коэффициенты характеристического многочлена: ", round.(coefficients, digits=6))
    poly = Polynomial(reverse(coefficients))
    
    # Находим корни (собственные значения)
    λs = roots(poly)

    B = B_total
    n = size(B, 1)
    eigenvectors = Vector{Vector{Float64}}()
    
    for λ in λs
        # Собственный вектор матрицы Фробениуса y
        # y = (λⁿ⁻¹, λⁿ⁻², ..., λ, 1)
        # из системы y₁ = λy₂, y₂ = λy₃, ..., y_{n-1} = λy_n, y_n = 1
        y = [λ^(n-i) for i in 1:n]
        
        # Собственный вектор исходной матрицы: x = B ⋅ y
        x = B * y
        
        # Нормировка собственного вектора
        x_norm = x / norm(x)
        push!(eigenvectors, x_norm)
    end
    
    return λs, eigenvectors
end

danilevsky_method_symmetric (generic function with 1 method)

In [10]:
function test_danilevski()
# Матрица из примера (симметричная)
    A = [2.2   1.0   0.5   2.0;
         1.0   1.3   2.0   1.0;
         0.5   2.0   0.5   1.6;
         2.0   1.0   1.6   2.0]
    
    println("Матрица A:")
    println(A)
    println()

    λs, vectors = danilevsky_method_symmetric(A)
    println("Собственные значения:", round.(λs, digits=4))
    println("Собственные векторы:")
    for (i, vec) in enumerate(vectors)
        println("  v$i = ", round.(vec, digits=4))
    end
    println()

    tol = 1e-12
    checks(A, λs, tol) 
end

test_danilevski()

Матрица A:
[2.2 1.0 0.5 2.0; 1.0 1.3 2.0 1.0; 0.5 2.0 0.5 1.6; 2.0 1.0 1.6 2.0]

Коэффициенты характеристического многочлена: [1.0, -6.0, -0.2, 12.735, -2.7616]
Собственные значения:[-1.4201, 0.2226, 1.5454, 5.652]
Собственные векторы:
  v1 = [-0.222, 0.5159, -0.7573, 0.3333]
  v2 = [-0.5219, -0.4549, 0.1534, 0.7051]
  v3 = [0.6289, -0.5726, -0.4857, 0.2019]
  v4 = [0.5317, 0.4462, 0.4088, 0.5925]

λ1 = -1.420087 ∈ {Δ2=[-2.7, 5.3], Δ3=[-3.6, 4.6], Δ4=[-2.6, 6.6]}
λ2 = 0.222636 ∈ {Δ1=[-1.3, 5.7], Δ2=[-2.7, 5.3], Δ3=[-3.6, 4.6], Δ4=[-2.6, 6.6]}
λ3 = 1.545418 ∈ {Δ1=[-1.3, 5.7], Δ2=[-2.7, 5.3], Δ3=[-3.6, 4.6], Δ4=[-2.6, 6.6]}
λ4 = 5.652032 ∈ {Δ1=[-1.3, 5.7], Δ4=[-2.6, 6.6]}

Результаты проверок:
След матрицы: true
Определитель: true
Интервалы Гершгорина: true
Все проверки пройдены: true
